The following is an attempt to obtain additional information directly from the Wilson Archives, namely the creators and subjects.
The output of this is saved in extracted_data/compiled/telegrams_with_extracted_features.csv

In [1]:
import digitalarchive
korean_war_docs = digitalarchive.Document.match(description ="korean war").all()
print("total num documents", len(korean_war_docs))

total num documents 2665


In [ ]:
import html
import re

def clean_html(string):
    content = html.unescape(string)
    search = re.search("<span><p class=............>(.*)</p></span>", content)
    if search != None:
        body = re.sub("<.*?>", "", search.group(1))
        return body
    else:
        return ""

In [ ]:
def extract_sender_receiver_from_title(title):
  arr = title.split(" ")
  sender = ""
  receiver = ""
  for i in range(len(arr)):
    if arr[i] == "to":
      sender = arr[i-1]
      receiver = " ".join(arr[i+1:])
  return sender, receiver

In [ ]:
# Searches for valid translations and extracts information from the json raws directly into a nested list.
# Schema: title, date, from, to, body, creators, subjects

import json
telegrams = []

for doc in korean_war_docs:
    errors = 0
    try:
        doc.hydrate()
        jsondoc = json.loads(doc.json())
        translations = doc.translations
        if len(doc.type) > 0 and len(translations) > 0:
          if doc.type[0].name == 'Telegram':
            print("Valid document found:", jsondoc["title"])
            translation = translations[0]
            translation.hydrate()
            if translation.raw is not None:
              body = clean_html(translation.raw.decode('utf-8'))

              subjects = []
              for s in jsondoc["subjects"]:
                subjects.append(s["name"])

              creators = []
              for c in jsondoc["creators"]:
                creators.append(c["name"])
              
              item = {"title": jsondoc["title"], "date": jsondoc["doc_date"], "raw": translation.raw.decode('utf-8'), "body": body, "creators": creators, "subjects": subjects}
              telegrams.append(item)

    except:
        errors += 1
        continue

Valid document found: Telegram from Aleksandr Vasilevsky to Stalin
Valid document found: Cable from Aleksandr Vasilevsky to Stalin
Valid document found: Cable No. 121973, Meretskov and Shytkov to Cde. Stalin
Valid document found: Telephone Message via VCh, I. Stalin to Kim Il Sung
Valid document found: Telegram, Shtemenko and Vasilevsky to Stalin
Valid document found: Ciphered Telegram from Shtykov
Valid document found: Telegram, Shtykov to Stalin
Valid document found: Cable Telegram no. 59363 from Kovalev to Stalin, containing a message from Mao Zedong
Valid document found: Draft Reply to Mao Zedong's Telegram from Stalin via Molotov
Valid document found: Cable Telegram no. 20475 from Gromyko to Kovalev
Valid document found: Telegram to Mao Zedong from Nie Rongzhen concerning the Repatriation of Ethnic Korean Soldiers to North Korea
Valid document found: Cable, Liu Shaoqi to Chairman Mao [Zedong]
Valid document found: Outgoing Telegram No. 2429, Vyshinsky to Shtykov
Valid document fou

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(telegrams)
df

,title,date,body,creators,subjects
0,Telegram from Aleksandr Vasilevsky to Stalin,19450818,18.8.45Moscow. Personally to Cde. StalinCopy: ...,"[Vasilevsky, Aleksandr Mikhaylovich]","[World War, 1939-1945--Campaigns--Japan--Hokka..."
1,Cable from Aleksandr Vasilevsky to Stalin,19450820,Moscow. Cde. StalinCopy: General Staff Cde. An...,"[Vasilevsky, Aleksandr Mikhaylovich]","[Kuril Islands, World War, 1939-1945--Campaign..."
2,"Cable No. 121973, Meretskov and Shytkov to Cde...",19470512,[handwritten at the top: to Cde. StalinI think...,"[Shtykov, Terentiĭ Fomich, 1907-1964]",[Korea (North)--Foreign relations--Soviet Unio...
3,"Telephone Message via VCh, I. Stalin to Kim Il...",19481012,Telephone message via VChCorrected text of ate...,"[Stalin, Joseph, 1879-1953]",[Korea (North)--Foreign relations--Soviet Unio...
4,"Telegram, Shtemenko and Vasilevsky to Stalin",19490420,UNCLASSIFIEDCopy Nº 1[handwritten: from Cdes. ...,"[Vasilevsky, Aleksandr Mikhaylovich, Shtemenko...","[38th Parallel, Korea (North)--Foreign relatio..."
...,...,...,...,...,...
372,"Hungarian Embassy in Mongolia, Report, 2 March...",19830302,"On February 23rd, Vietnamese Ambassador Cao Ki...",[],"[Non-Aligned Movement, China--Foreign relation..."
373,"Hungarian Embassy in the DPRK, Ciphered Telegr...",19830815,The Cuban ambassador informed me about their p...,[],"[Non-Aligned Movement, Cuba--Foreign relations..."
374,"Hungarian Embassy in India, Ciphered Telegram,...",19830928,"Comrade Lahiri, the deputy head of the Interna...",[],"[Communist Party of India, India--Foreign rela..."
375,"Hungarian Embassy in Ethiopia, Ciphered Telegr...",19851126,"In recent times, the strengthening relationshi...",[],"[Ethiopia--Foreign relations--Soviet Union, Et..."
